<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/S_Vs_T_LSTM_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
!pip install bert-for-tf2

In [2]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os, re
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
os.environ['KAGGLE_KEY'] = "001b3a30170775e55950edb6ff0c9b17"
!kaggle datasets download -d nikhileswarkomati/suicide-watch
!unzip '/content/suicide-watch.zip'

 99% 114M/115M [00:01<00:00, 75.2MB/s]
100% 115M/115M [00:01<00:00, 91.7MB/s]
Archive:  /content/suicide-watch.zip
  inflating: SuicideAndDepression_Detection.csv  


In [4]:
data = pd.read_csv('/content/SuicideAndDepression_Detection.csv', lineterminator = '\n')
data.sample(5)

,text,class
284641,"For the first time, I saw a serious post witho...",teenagers
294826,My English teacher asked to give my thought ab...,teenagers
200658,Whats r/5050? Title says it all. What is it?\n...,teenagers
122067,my mum has sent me the most passive aggressive...,teenagers
184336,she didnt leave you on read king you left her ...,teenagers


In [5]:
print(data.shape)
data['class'].value_counts()

(348111, 2)


depression      116037
teenagers       116037
SuicideWatch    116037
Name: class, dtype: int64

In [ ]:
def preprocess(string):
    phrase = str(string)
    phrase = re.sub('[^a-z]+', ' ', phrase, flags = re.IGNORECASE)
    phrase = re.sub('(\s+)', ' ', phrase)
    phrase = re.sub('http\S+', ' ', phrase)
    phrase = phrase.lower()
    words_li = ['filler']
    li = list(stopwords.words()) + words_li
    text_tokens = word_tokenize(phrase)
    return " ".join([word for word in text_tokens if word not in li])
print(data['text'].apply(lambda x: len(x.split(' '))).sum())

59527144


In [ ]:
data['text'] = data['text'].map(lambda string: preprocess(string))

In [ ]:
print(data['text'].apply(lambda x: len(x.split(' '))).sum())

59527144


In [ ]:
data.loc[(data['class'] != 'depression'), 'text'][:1000].shape

(1000,)

In [6]:
# X = data.iloc[:, 0].values
# y = data.iloc[:, 1].values

# le = LabelEncoder()

# y = le.fit_transform(y)

# train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
# train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
# print(train_X.shape, test_y.shape, val_X.shape)

#------------------------------------------------------------------------------------------
X = data.loc[(data['class'] != 'depression'), 'text'].values
y = data.loc[(data['class'] != 'depression'), 'class'].values

le = LabelEncoder()

y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
print(train_X.shape, test_y.shape, val_X.shape)

(23207,) (116037,) (92830,)


In [14]:
from keras.layers import LSTM, Conv1D, MaxPooling1D
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint

from keras.layers import LSTM
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import itertools


%matplotlib inline


from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.preprocessing import text, sequence
from keras import utils


embedding_vecor_length = 32
vocab_size = 10000
max_length = 600
oov_tok = '<OOV>'
trunc_type = 'post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_X)
padded = sequence.pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length = max_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout = 0.3, recurrent_dropout = 0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_lstm.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
callbacks_list = [checkpoint, earlystopping]
model.fit(padded, train_y, epochs=10, batch_size=256,verbose = 1,callbacks = callbacks_list,validation_split = 0.1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           320000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 600, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 300, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 376,405
Trainable params: 376,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
82/82 [==============================] - 148s 2s/step - loss: 0.5759 - accuracy: 0.6806 - val_loss

In [17]:
from keras.models import load_model
model = load_model('/content/model_lstm.h5')
testing_sequences = tokenizer.texts_to_sequences(test_X)
testing_padded = sequence.pad_sequences(testing_sequences, maxlen = max_length)
y_pred = model.predict(testing_padded)
print(accuracy_score(np.rint(y_pred), test_y))
print(classification_report(np.rint(y_pred), test_y))

0.9265837620758895
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.93     56553
         1.0       0.94      0.92      0.93     59484

    accuracy                           0.93    116037
   macro avg       0.93      0.93      0.93    116037
weighted avg       0.93      0.93      0.93    116037

